In [ ]:
import tweepy
import os
import re
import logging
import nltk
import string
import datetime
import pandas as pd
import numpy as np

!pip install preprocessor

import preprocessor as p
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from nltk.corpus import stopwords
from textblob import TextBlob




  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp36-none-any.whl size=4477 sha256=a062a36bba0ad5d4debcde84470f234a059c3a8d5fbc1d596f3a16e87c387b36
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor


In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

#import fire

import sys
import os
import json
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns

# to view all columns
pd.set_option("display.max.columns", None)

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

!pip install preprocessor
import preprocessor as p

In [ ]:
import twitter_credentials



consumer_key = twitter_credentials.CONSUMER_KEY
consumer_secret = twitter_credentials.CONSUMER_SECRET
access_token = twitter_credentials.ACCES_TOKEN
access_token_secret = twitter_credentials.ACCES_TOKEN_SECRET

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth, wait_on_rate_limit=True)

#places = auth_api.geo_search(query="KENYA", granularity="country")
#place_id = places[0].id
#places

In [ ]:
handles=pd.read_csv('handles.csv')
handles['0']

0         IAQsusan
1     thetechsamya
2     SeeTheAirBot
3     elonfantommy
4      INFORSE_org
          ...     
78       ESusWatch
79    membaibrahim
80    Roby41262525
81       Eco1stArt
82     EstherMulin
Name: 0, Length: 83, dtype: object

In [ ]:
def get_handle_tweets(self, handles:list=[], items_count=20):
        self.handles = handles
        
        if len(self.handles) > 0: 
            for handle in self.handles:
                print(f"collecting tweets of -> {handle}")
                users_tweets={}
                # this helps avoid Tweepy errors like suspended users or user not found errors
                try: 
                    item = self.auth_api.get_user(handle)
                except tweepy.TweepError as e:
                    print("found errors!!!")
                    continue
                 
                #check if handle is a potential bot    
                if self.check_is_bot(handle):
                    print('bot alert!!!, skipping the bad guy :(')
                    continue
                else:
                    current_handle_tweets = Cursor(api.user_timeline, id=handle,
                                                        tweet_mode='extended',
                                                        lang="en").items(items_count)
                users_tweets = {} 
                for tweet in current_handle_tweets:
                    users_tweets[handle] = ({'tweet_text':tweet.full_text.encode('utf-8'),
                                             'tweet_date':tweet._json['created_at'],
                                            'retweet_count':tweet._json['retweet_count'],
                                            'favorite_count':tweet._json['favorite_count']})
        
        self.handles_data = pd.DataFrame(users_tweets).T
            
        return self.handles_data

In [ ]:
data=get_handle_tweets(handles['0'])

UnboundLocalError: ignored

In [ ]:

import time
class tweetsearch():
    '''
    This is a basic class to search and download twitter data.
    You can build up on it to extend the functionalities for more 
    sophisticated analysis
    '''
    def __init__(self, auth=None):
            
        if auth is None:
            
            #Variables that contains the user credentials to access Twitter API 
            #consumer_key = os.environ.get('TWITTER_API_KEY')
            #consumer_secret = os.environ.get('TWITTER_API_SECRET')
            #access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
            #access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')
            


            #This handles Twitter authetification and the connection to Twitter Streaming API
            #auth = OAuthHandler(consumer_key, consumer_secret)
            #auth.set_access_token(access_token, access_token_secret)
            auth=OAuthHandler(twitter_credentials.CONSUMER_KEY,twitter_credentials.CONSUMER_SECRET)
            auth.set_access_token(twitter_credentials.ACCES_TOKEN,twitter_credentials.ACCES_TOKEN_SECRET)
            

       
            

        #            
        self.auth = auth
        self.api = tweepy.API(self.auth,wait_on_rate_limit=True) 
    def cleanTweets(self, twitterText):
        #logger.info("Cleaning a tweet \n\n")
        tweet = p.clean(twitterText)

        # Happy Emoticons
        emoticonsHappy = set([
            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
            '<3'])

        # Sad Emoticons
        emoticonsSad = set([
            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
            ':c', ':{', '>:\\', ';('])

        # Emoji patterns
        emojiPattern = re.compile("[" u"\U0001F600-\U0001F64F"  # emoticons
                                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                      u"\U00002702-\U000027B0"
                                      u"\U000024C2-\U0001F251" "]+", flags=re.UNICODE)

        # Combine the sad and happy emoticons
        emoticons = emoticonsHappy.union(emoticonsSad)

        # tokenize the words in the tweet and get the stop words
        stopWords = set(stopwords.words('english'))
        wordTokens = nltk.word_tokenize(tweet)

        # after tweepy preprocessing the colon symbol left remain after
        # removing mentions
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)

        # replace consecutive non-ASCII characters with a space
        tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)

        # remove emojis from tweet
        tweet = emojiPattern.sub(r'', tweet)

        # looping through conditions
        filteredTweet = []
        for w in wordTokens:
            # check tokens against stop words , emoticons and punctuations
            if w not in stopWords and w not in emoticons and w not in string.punctuation:
                filteredTweet.append(w)

        return ' '.join(filteredTweet)


    def get_tweets(self,hashtag,geocode,csvfile=None):
        
        #max_id= Returns results with an ID less than(that is,older than)or eual to the specified ID
        #since_id= Returns results with an ID greater than(that is,more recent than)the specified ID
        
        #backoff_counter=1    
        #while True:
        #   try:
            tweets = tweepy.Cursor(auth_api.search,q=hashtag,geocode=geocode, since_id ='2020-04-01' ).items(10)
            #,monitor_rate_limit=True, wait_on_rate_limit=True,wait_on_rate_limit_notify = True,retry_count = 5, #retry 5 times
                      # retry_delay = 5, #seconds to wait for retry
                      
                                                                                                      
                # geocode="-1.286389, 36.817223,40km",

            users = []
            #followers = []
            #friends = []
            #description = []
            #Tweets = []
            #blob=[]
            #polarity=[]
            #subjectivity=[]


            for status in tweets:
              name = status.user.screen_name
              desc = status.user.description

                    #clean the tweet
                    #Tweet = self.cleanTweets(status.text)
              #Tweet=status.text

                    #Calculate the sentiment
              #blob=TextBlob(Tweet)
              #polarity=blob.sentiment.polarity
              #subjectivity=blob.sentiment.subjectivity

              #following = status.user.friends_count
              #follower = status.user.followers_count
              users.append(name)
              #followers.append(follower)
              #friends.append(following)
              #description.append(desc)
              #Tweets.append(Tweet) 

                    
            data = {'screen_name':users}
                    #'followers':followers, 'friends':friends, 'description':description, 'tweets':Tweets}
            tweets_df = pd.DataFrame(data)
                                                  
            #except tweepy.TweepError as e:
             #   print(e.reason)
             #   time.sleep(60*backoff_counter)
             #   backoff_counter += 1
              #  continue
        #     df['screen_name'] = users
        #     df['location'] = location

            if not csvfile is None:
            #save it to file
              tweets_df.to_csv(csvfile)
      
            return tweets_df

    def get_handles(self,handles:list=[]):
            self.handles = handles
            users_tweets={}
            if len(self.handles)>0:
                for handle in self.handles:
                  print(f"collecting tweets of -> {handle}")
                     
                  # this helps avoid Tweepy errors like suspended users or user not found errors
                  try: 
                      item = self.api.get_user(handle)
                  except tweepy.TweepError as e:
                      print("found errors!!!")
                      continue
                  
                #check if handle is a potential bot    
                #if self.check_is_bot(handle):
                  #    print('bot alert!!!, skipping the bad guy :(')
                  #    continue
                  #else:
                  current_handle_tweets = Cursor(self.api.user_timeline, id=handle,
                                                                      tweet_mode='extended',
                                                                      lang="en").items(20)
                   
                  for tweet in current_handle_tweets:
                      users_tweets[handle] = ({'tweet_text':tweet.full_text.encode('utf-8'),
                                                          'tweet_date':tweet._json['created_at'],
                                                          'retweet_count':tweet._json['retweet_count'],
                                                          'favorite_count':tweet._json['favorite_count']})
                        
            self.handles_data = pd.DataFrame(users_tweets).T            
            return self.handles_data

In [ ]:
ts=tweetsearch()
df=ts.get_handles(handles['0'])

collecting tweets of -> IAQsusan
collecting tweets of -> thetechsamya
collecting tweets of -> SeeTheAirBot
collecting tweets of -> elonfantommy
collecting tweets of -> INFORSE_org
collecting tweets of -> ESusWatch
collecting tweets of -> AfricaSustain1
collecting tweets of -> SeeTheAirBot
collecting tweets of -> elonfantommy
collecting tweets of -> solhog
collecting tweets of -> RichardMunang
collecting tweets of -> CodesmithSys
collecting tweets of -> MusyokiGrace
collecting tweets of -> artspace254
collecting tweets of -> InnoDeckoks
collecting tweets of -> agomagroupltd
collecting tweets of -> MukiibiEmma2
collecting tweets of -> VilmaOnyamasi
collecting tweets of -> EricDNjuguna
collecting tweets of -> RGabdho
collecting tweets of -> _monicah_shiku
collecting tweets of -> estate_rex
collecting tweets of -> estate_rex
collecting tweets of -> HenryOgoti
collecting tweets of -> HenryOgoti
collecting tweets of -> EugeneMwangangi
collecting tweets of -> KenyaNeoEngLtd
collecting tweets 

In [ ]:
df.to_csv('tweets.csv')

In [ ]:
def get_handles(self,handles:list=[]):
  self.handles = handles
   
  if len(self.handles)>0:
      for handle in self.handles:
        print(f"collecting tweets of -> {handle}")
            
        # this helps avoid Tweepy errors like suspended users or user not found errors
        try: 
            item = self.auth_api.get_user(handle)
        except tweepy.TweepError as e:
            print("found errors!!!")
            continue
         
      #check if handle is a potential bot    
        if self.check_is_bot(handle):
            print('bot alert!!!, skipping the bad guy :(')
            continue
        else:
            current_handle_tweets = Cursor(api.user_timeline, id=handle,
                                                            tweet_mode='extended',
                                                            lang="en").items(items_count)
        users_tweets={}    
        for tweet in current_handle_tweets:
              users_tweets[handle] = ({'tweet_text':tweet.full_text.encode('utf-8'),
                                                'tweet_date':tweet._json['created_at'],
                                                'retweet_count':tweet._json['retweet_count'],
                                                'favorite_count':tweet._json['favorite_count']})
              
  self.handles_data = pd.DataFrame(users_tweets).T
              
  return self.handles_data

In [ ]:
data=get_handles(handles['0'])
data

UnboundLocalError: ignored

In [ ]:
hashtags=["#BreatheLife","#AirPollution","#airquality","#cleanair","#airpollution","#pollution","#hvac","#airpurifier" ,"#indoorairquality","#air","#climatechange","#indoorair","#environment" ,"#airconditioning","#heating" , "#freshair", "#airfilter","#ventilation","#airconditioner","#airqualityindex", "#pm2_5 ","#emissions","#natureishealing","#nature","#pollutionfree" ,"#wearethevirus",'AirPollution', 'Environment', 'Ozone Layer', 'Global Warming', 'Climate Change', 'Greenhouse Gases', 'Trees', 'Carbon',
        'Aerosals', 'Air', 'Save the planet', 'Factories', 'Hygroscopicity', 'Inversion', 'Sulfur', 'AIRS', 'ecosystem', 'Hydrochlorofluorocarbon',
        'hydrocarbon', 'TAC', 'zero', 'pollutant', '#air', '#pollution', '#airpollution', '#coal', '#particles', '#smog', '#cleanair',
       '#airqualityindex', '#climatechange', '#airquality', '#globalwarming', '#airpollutionawareness', '#airpollutioncontrol',
       '#CleanEnergy', '#saveearth']

geocodes=["0.0236,37.9062,530km", "9.0820,8.6753, 750km",  "30.5595, 22.9375, 1000km", "1.3733, 32.2903, 400km"]


names = []


names = []
for hashtag in hashtags:
    tweets = tweepy.Cursor(auth_api.search, q = hashtag ,geocode="0.0236,37.9062,530km",monitor_rate_limit=True, wait_on_rate_limit=True,
                          wait_on_rate_limit_notify = True,retry_count = 5, 
                       retry_delay = 5).items(1000000)
    users = []
    for status in tweets:
        name = status.user.screen_name
        t=status.text
        users.append(name)
    names.append(users)
screen_names = [y for x in names for y in x]
screen_names

Rate limit reached. Sleeping for: 843
Rate limit reached. Sleeping for: 857
Rate limit reached. Sleeping for: 853
Rate limit reached. Sleeping for: 858
Rate limit reached. Sleeping for: 856
Rate limit reached. Sleeping for: 853
Rate limit reached. Sleeping for: 857
Rate limit reached. Sleeping for: 855
Rate limit reached. Sleeping for: 856
Rate limit reached. Sleeping for: 855
Rate limit reached. Sleeping for: 858
Rate limit reached. Sleeping for: 857
Rate limit reached. Sleeping for: 859


['IAQsusan',
 'thetechsamya',
 'SeeTheAirBot',
 'elonfantommy',
 'prothero_james',
 'KlaudineWakasa',
 'IAQsusan',
 'innocentmaloba',
 'thetechsamya',
 'KingOwenR',
 'Kampala4Climate',
 'INFORSE_org',
 'ESusWatch',
 'Stephen91235019',
 'DSS_Lab',
 'h_doukas',
 'KingOwenR',
 'Kampala4Climate',
 'AfricaSustain1',
 'SeeTheAirBot',
 'elonfantommy',
 'prothero_james',
 'KlaudineWakasa',
 'IAQsusan',
 'innocentmaloba',
 'thetechsamya',
 'KingOwenR',
 'Kampala4Climate',
 'solhog',
 'RichardMunang',
 'JimmyKarakaria',
 'jovs_98',
 'GLOBE_intl',
 'SaniGamboUmar1',
 'WasteOrCreate',
 'NagudiIrene',
 'DarioMelian',
 'RSEOnline',
 'RobertoValentUN',
 'UNEP_NAmerica',
 'svilardyq',
 'SAMantilla1',
 'SaleemulHuq',
 'SaveWildlife12',
 'kuepouo',
 'dahirualiyu099',
 'Shenever_knew',
 'UrgeCollective',
 'poornimathreya',
 'YS4CIN',
 'OlumideIDOWU',
 'ClimateWed',
 'CarolynFahm',
 'ejvhattum',
 'EU_ENV',
 'TLC_CN',
 'JTpaleo',
 'UNEP',
 'savinacp',
 'MacondoChao',
 'BVEdgerton',
 'tuula',
 'CUGHnews',
 

In [ ]:
screen_names


NameError: ignored

In [ ]:

import time
class tweetsearch():
    '''
    This is a basic class to search and download twitter data.
    You can build up on it to extend the functionalities for more 
    sophisticated analysis
    '''
    def __init__(self, auth=None):
            
        if auth is None:
            
            #Variables that contains the user credentials to access Twitter API 
            #consumer_key = os.environ.get('TWITTER_API_KEY')
            #consumer_secret = os.environ.get('TWITTER_API_SECRET')
            #access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
            #access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')
            


            #This handles Twitter authetification and the connection to Twitter Streaming API
            #auth = OAuthHandler(consumer_key, consumer_secret)
            #auth.set_access_token(access_token, access_token_secret)
            auth=OAuthHandler(twitter_credentials.CONSUMER_KEY,twitter_credentials.CONSUMER_SECRET)
            auth.set_access_token(twitter_credentials.ACCES_TOKEN,twitter_credentials.ACCES_TOKEN_SECRET)
            

       
            

        #            
        self.auth = auth
        self.api = tweepy.API(self.auth,wait_on_rate_limit=True) 
    def cleanTweets(self, twitterText):
        #logger.info("Cleaning a tweet \n\n")
        tweet = p.clean(twitterText)

        # Happy Emoticons
        emoticonsHappy = set([
            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
            '<3'])

        # Sad Emoticons
        emoticonsSad = set([
            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
            ':c', ':{', '>:\\', ';('])

        # Emoji patterns
        emojiPattern = re.compile("[" u"\U0001F600-\U0001F64F"  # emoticons
                                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                      u"\U00002702-\U000027B0"
                                      u"\U000024C2-\U0001F251" "]+", flags=re.UNICODE)

        # Combine the sad and happy emoticons
        emoticons = emoticonsHappy.union(emoticonsSad)

        # tokenize the words in the tweet and get the stop words
        stopWords = set(stopwords.words('english'))
        wordTokens = nltk.word_tokenize(tweet)

        # after tweepy preprocessing the colon symbol left remain after
        # removing mentions
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)

        # replace consecutive non-ASCII characters with a space
        tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)

        # remove emojis from tweet
        tweet = emojiPattern.sub(r'', tweet)

        # looping through conditions
        filteredTweet = []
        for w in wordTokens:
            # check tokens against stop words , emoticons and punctuations
            if w not in stopWords and w not in emoticons and w not in string.punctuation:
                filteredTweet.append(w)

        return ' '.join(filteredTweet)


    def get_tweets(self,hashtag,geocode,csvfile=None):
        
        #max_id= Returns results with an ID less than(that is,older than)or eual to the specified ID
        #since_id= Returns results with an ID greater than(that is,more recent than)the specified ID
        
        #backoff_counter=1    
        #while True:
        #   try:
            tweets = tweepy.Cursor(auth_api.search,q=hashtag,geocode=geocode, since_id ='2020-04-01' ).items(10)
            #,monitor_rate_limit=True, wait_on_rate_limit=True,wait_on_rate_limit_notify = True,retry_count = 5, #retry 5 times
                      # retry_delay = 5, #seconds to wait for retry
                      
                                                                                                      
                # geocode="-1.286389, 36.817223,40km",

            users = []
            #followers = []
            #friends = []
            #description = []
            #Tweets = []
            #blob=[]
            #polarity=[]
            #subjectivity=[]


            for status in tweets:
              name = status.user.screen_name
              desc = status.user.description

                    #clean the tweet
                    #Tweet = self.cleanTweets(status.text)
              #Tweet=status.text

                    #Calculate the sentiment
              #blob=TextBlob(Tweet)
              #polarity=blob.sentiment.polarity
              #subjectivity=blob.sentiment.subjectivity

              #following = status.user.friends_count
              #follower = status.user.followers_count
              users.append(name)
              #followers.append(follower)
              #friends.append(following)
              #description.append(desc)
              #Tweets.append(Tweet) 

                    
            data = {'screen_name':users}
                    #'followers':followers, 'friends':friends, 'description':description, 'tweets':Tweets}
            tweets_df = pd.DataFrame(data)
                                                  
            #except tweepy.TweepError as e:
             #   print(e.reason)
             #   time.sleep(60*backoff_counter)
             #   backoff_counter += 1
              #  continue
        #     df['screen_name'] = users
        #     df['location'] = location

            if not csvfile is None:
            #save it to file
              tweets_df.to_csv(csvfile)
      
            return tweets_df

In [ ]:
hashtags=["#BreatheLife","#AirPollution","#airquality","#cleanair","#airpollution","#pollution","#hvac","#airpurifier" ,"#indoorairquality","#air","#climatechange","#indoorair","#environment" ,"#airconditioning","#heating" , "#freshair", "#airfilter","#ventilation","#airconditioner","#airqualityindex", "#pm2_5 ","#emissions","#natureishealing","#nature","#pollutionfree" ,"#wearethevirus",'AirPollution', 'Environment', 'Ozone Layer', 'Global Warming', 'Climate Change', 'Greenhouse Gases', 'Trees', 'Carbon',
        'Aerosals', 'Air', 'Save the planet', 'Factories', 'Hygroscopicity', 'Inversion', 'Sulfur', 'AIRS', 'ecosystem', 'Hydrochlorofluorocarbon']
        #'hydrocarbon', 'TAC', 'zero', 'pollutant', '#air', '#pollution', '#airpollution', '#coal', '#particles', '#smog', '#cleanair',
       #'#airqualityindex', '#climatechange', '#airquality', '#globalwarming', '#airpollutionawareness', '#airpollutioncontrol',
       #'#CleanEnergy', '#saveearth']

#geocodes=["33.918861, 18.423300,40km"," -26.205681, 28.046822,40km","5.550000, -0.020000,40km","-1.286389, 36.817223,40km","-4.043740, 39.658871,40km","6.465422, 3.406448,40km","-1.935114, 30.082111,40km"," 0.347596, 32.582520,40km"]
geocodes=["0.0236,37.9062,530km", "9.0820,8.6753, 750km",  "30.5595, 22.9375, 1000km", "1.3733, 32.2903, 400km"]
#geocodes=["6.48937,3.37709,500km","-33.99268,18.46654,500km","-26.22081,28.03239,500km","5.58445,-0.20514,500km","-1.27467,36.81178,500km","-4.04549,39.66644,500km","-1.95360,30.09186,500km","0.32400,32.58662,500km"]



ts = tweetsearch()
for hashtag in hashtags:
  for geocode in geocodes:
    df = ts.get_tweets(hashtag,geocode) 
    print(hashtag)
    print(geocode)

#BreatheLife
0.0236,37.9062,530km
#BreatheLife
9.0820,8.6753, 750km
#BreatheLife
30.5595, 22.9375, 1000km
#BreatheLife
1.3733, 32.2903, 400km
#AirPollution
0.0236,37.9062,530km
#AirPollution
9.0820,8.6753, 750km
#AirPollution
30.5595, 22.9375, 1000km
#AirPollution
1.3733, 32.2903, 400km
#airquality
0.0236,37.9062,530km
#airquality
9.0820,8.6753, 750km
#airquality
30.5595, 22.9375, 1000km
#airquality
1.3733, 32.2903, 400km
#cleanair
0.0236,37.9062,530km
#cleanair
9.0820,8.6753, 750km
#cleanair
30.5595, 22.9375, 1000km
#cleanair
1.3733, 32.2903, 400km
#airpollution
0.0236,37.9062,530km
#airpollution
9.0820,8.6753, 750km
#airpollution
30.5595, 22.9375, 1000km
#airpollution
1.3733, 32.2903, 400km
#pollution
0.0236,37.9062,530km
#pollution
9.0820,8.6753, 750km
#pollution
30.5595, 22.9375, 1000km
#pollution
1.3733, 32.2903, 400km
#hvac
0.0236,37.9062,530km
#hvac
9.0820,8.6753, 750km
#hvac
30.5595, 22.9375, 1000km
#hvac
1.3733, 32.2903, 400km
#airpurifier
0.0236,37.9062,530km
#airpurifier
9.0

In [ ]:
df


,screen_name


In [ ]:
!pip install GetOldTweets3

In [ ]:
import GetOldTweets3 as got

for hashtag in hashtags:
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(hashtag)\
                                              .setSince("2015-09-10")\
                                              .setUntil("2016-01-01")\
                                              .setMaxTweets(200)\
                                              .setEmoji("unicode")
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
    print(tweet.text)

 
 
 




,screen_name,followers,friends,description,tweets
0,AHAM_Voice,1641,1530,The Association of Home Appliance Manufacturer...,Want to improve your indoor #AirQuality? Get a...
1,forzagaribaldi,1079,1814,I like Neopan and the smell of fixer in the mo...,RT @CleanAirUK: Really excellent item on BBC R...
2,MorganAshHouTx,80,980,Public Health Communication Professional - For...,@HoustonHealth joins other city and county par...
3,SVNIR,394,93,,ARTICLE: The Importance of Indoor Air Quality ...
4,WeAre4CleanAir,672,412,The national association of 155 state & local ...,RT @LouAPCD: Air Quality is forecast to be Goo...
